### Testing Serp API for scraping google play reviews

In [84]:
import os
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
load_dotenv()
import serpapi

api_key = os.getenv("SERPAPI_KEY")
client = serpapi.Client(api_key=api_key)


In [85]:
results = client.search(
    engine = "google_play_product",
    product_id = "in.swiggy.android",
    store = "apps",
    all_reviews = "true",
    num = 199,
    sort_by = 2,
    json_restrictor = "reviews, serpapi_pagination",
)
#print (results)
#print ("Total Reviews: ", len(results["reviews"]))

In [86]:
results

{
    "reviews": [
        {
            "id": "168cf475-5b24-4f84-b91f-df256324c7de",
            "title": "Ravlin Kour",
            "avatar": "https://play-lh.googleusercontent.com/a/ACg8ocJB3V7t9wjgxUQmrqW9OVYW5LHSty4ZDlJYTsHrxO4m1FUP8w=mo",
            "rating": 5.0,
            "snippet": "gud",
            "likes": 0,
            "date": "January 07, 2026",
            "iso_date": "2026-01-07T05:26:10Z",
            "response": {
                "title": "Swiggy",
                "snippet": "Hi there, thank you for your patronage and the perfect rating. We will strive to provide consistent seamless ordering experiences. Keep Swiggying!",
                "date": "January 07, 2026",
                "iso_date": "2026-01-07T05:32:31Z"
            }
        },
        {
            "id": "8be1730f-ee99-4c98-bf08-366d8f622cae",
            "title": "FREE FIRE FIRE",
            "avatar": "https://play-lh.googleusercontent.com/a-/ALV-UjXvXD1K37vgU6l5zK1qqp-si2_U9x4YyR3xBQIEyUYO9feEN8U"

In [87]:
import json
with open('results.json', 'w') as f:
    json.dump(results["reviews"], f, indent=4)
#results

In [88]:
df = pd.DataFrame(columns=["Date", "Review"])
for item in results["reviews"]:
    new_row = {'Date': item["iso_date"], 'Review': item["snippet"]}
    df.loc[len(df)] = new_row
    #df = pd.DataFrame({"date": item["iso_date"], "review": item["snippet"]})
print(df)

                     Date          Review
0    2026-01-07T05:26:10Z             gud
1    2026-01-07T05:24:29Z        very sad
2    2026-01-07T04:58:37Z  good very good
3    2026-01-07T04:51:54Z     perfect 👍👍👍
4    2026-01-07T04:48:55Z       achha app
..                    ...             ...
194  2026-01-06T14:39:30Z        best one
195  2026-01-06T14:39:17Z      fraudsters
196  2026-01-06T14:36:45Z            good
197  2026-01-06T14:36:05Z            good
198  2026-01-06T14:35:08Z            good

[199 rows x 2 columns]


In [89]:
def filter_reviews_by_date(data, start_date, end_date):
    """
    Filter reviews by date range and return a DataFrame with Date and Review columns.
    
    Parameters:
    -----------
    data : list of dict
        List of review dictionaries containing 'iso_date' and 'snippet' fields
    start_date : str or datetime
        Start date in format 'YYYY-MM-DD' or datetime object
    end_date : str or datetime
        End date in format 'YYYY-MM-DD' or datetime object
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with 'Date' and 'Review' columns filtered by date range
    """
    # Convert string dates to datetime objects if needed
    if isinstance(start_date, str):
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
    if isinstance(end_date, str):
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
    
    # Extract date and review from data
    filtered_data = []
    
    for review in data:
        # Parse the iso_date string to datetime
        #review_date = datetime.fromisoformat(review['iso_date'].replace('Z', '+00:00'))
        review_date = pd.to_datetime(review['date'], errors='coerce')
        #print (review_date)
        
        # Check if the review date falls within the range (comparing dates only)
        review_date_only = review_date.date()
        start_date_only = start_date.date() if hasattr(start_date, 'date') else start_date
        end_date_only = end_date.date() if hasattr(end_date, 'date') else end_date
        
        if start_date_only <= review_date_only <= end_date_only:
            filtered_data.append({
                'Date': review_date_only,
                'Review': review['snippet']
            })
    
    # Create DataFrame
    df = pd.DataFrame(filtered_data)

    # Sort by Date in descending order (latest first)
    if not df.empty:
        df = df.sort_values(by='Date', ascending=False).reset_index(drop=True)
    
    return df

In [90]:
result_df = filter_reviews_by_date(results["reviews"], '2026-01-01', '2026-01-08')
print(result_df)

           Date                                             Review
0    2026-01-07                                                gud
1    2026-01-07         It was a mistake to take its subscription.
2    2026-01-07                                           very sad
3    2026-01-07                                            good to
4    2026-01-07  Frequent order mistakes, unhelpful customer su...
..          ...                                                ...
194  2026-01-06                                               good
195  2026-01-06                                       extra charge
196  2026-01-06                                      thanks swiggy
197  2026-01-06              Lakhimpur walo iss app ka use na krna
198  2026-01-06                                               good

[199 rows x 2 columns]


Pagination

In [ ]:
while "serpapi_pagination" in results:
    results = client.search(
        engine = "google_play_product",
        product_id = "in.swiggy.android",
        store = "apps",
        all_reviews = True,
        num = 199,
        sort_by = 2,
        json_restrictor = "reviews, serpapi_pagination",
        next_page_token = results["serpapi_pagination"]["next_page_token"],    
    )

    
    
    print ("Total Reviews: ", len(results["reviews"]))

In [91]:
df = pd.DataFrame(columns=["Date", "Review"])
START_DATE = '2025-12-31'
END_DATE = '2026-01-08'
count = 0
while "serpapi_pagination" in results:
    results = client.search(
        engine = "google_play_product",
        product_id = "in.swiggy.android",
        store = "apps",
        all_reviews = True,
        num = 199,
        sort_by = 2,
        json_restrictor = "reviews, serpapi_pagination",
        next_page_token = results["serpapi_pagination"]["next_page_token"],    
    )

    temp_df = filter_reviews_by_date(results["reviews"], START_DATE, END_DATE)
    if len(temp_df)==0:
        break
    df = pd.concat ([df, temp_df], ignore_index=True)

    # Sort by Date in descending order
    if not df.empty:
        df = df.sort_values(by='Date', ascending=False).reset_index(drop=True)

    #print ("Total Reviews: ", len(results["reviews"]))
    count+=1
    print("Total Batches Fetched: ", count)
    
   

Total Batches Fetched:  1
Total Batches Fetched:  2
Total Batches Fetched:  3
Total Batches Fetched:  4
Total Batches Fetched:  5
Total Batches Fetched:  6
Total Batches Fetched:  7
Total Batches Fetched:  8
Total Batches Fetched:  9
Total Batches Fetched:  10
Total Batches Fetched:  11
Total Batches Fetched:  12
Total Batches Fetched:  13
Total Batches Fetched:  14
Total Batches Fetched:  15
Total Batches Fetched:  16
Total Batches Fetched:  17
Total Batches Fetched:  18
Total Batches Fetched:  19
Total Batches Fetched:  20


In [92]:
df

,Date,Review
0,2026-01-06,osm
1,2026-01-06,nice 👍
2,2026-01-06,excellent 👌
3,2026-01-06,super app
4,2026-01-06,"The worst app. Constant delays, constantly mis..."
...,...,...
3901,2025-12-31,Good OFFER
3902,2025-12-31,best service
3903,2025-12-31,There's a clear discrepancy in distance calcul...
3904,2025-12-31,I was having problems with zomato so I tried t...


### Final Function

In [98]:
def filter_reviews_by_date(data, start_date, end_date):
    """
    Filter reviews by date range and return a DataFrame with Date and Review columns.
    
    Parameters:
    -----------
    data : list of dict
        List of review dictionaries containing 'iso_date' and 'snippet' fields
    start_date : str or datetime
        Start date in format 'YYYY-MM-DD' or datetime object
    end_date : str or datetime
        End date in format 'YYYY-MM-DD' or datetime object
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with 'Date' and 'Review' columns filtered by date range
    """
    # Convert string dates to datetime objects if needed
    if isinstance(start_date, str):
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
    if isinstance(end_date, str):
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
    
    # Extract date and review from data
    filtered_data = []
    
    for review in data:
        # Parse the iso_date string to datetime
        #review_date = datetime.fromisoformat(review['iso_date'].replace('Z', '+00:00'))
        review_date = pd.to_datetime(review['date'], errors='coerce')
        #print (review_date)
        
        # Check if the review date falls within the range (comparing dates only)
        review_date_only = review_date.date()
        start_date_only = start_date.date() if hasattr(start_date, 'date') else start_date
        end_date_only = end_date.date() if hasattr(end_date, 'date') else end_date
        
        if start_date_only <= review_date_only <= end_date_only:
            filtered_data.append({
                'Date': review_date_only,
                'Review': review['snippet']
            })
    
    # Create DataFrame
    df = pd.DataFrame(filtered_data)

    # Sort by Date in descending order (latest first)
    if not df.empty:
        df = df.sort_values(by='Date', ascending=False).reset_index(drop=True)
    
    return df


def fetch_reviews(product_id, START_DATE, END_DATE):
    df = pd.DataFrame(columns=["Date", "Review"])

    results = client.search(
    engine = "google_play_product",
    product_id = "in.swiggy.android",
    store = "apps",
    all_reviews = "true",
    num = 199,
    sort_by = 2,
    json_restrictor = "reviews, serpapi_pagination",
    )
    

    while "serpapi_pagination" in results:

        temp_df = filter_reviews_by_date(results["reviews"], START_DATE, END_DATE)
        if len(temp_df)==0:
            break
        df = pd.concat ([df, temp_df], ignore_index=True)
        
        results = client.search(
            engine = "google_play_product",
            product_id = product_id,
            store = "apps",
            all_reviews = True,
            num = 199,
            sort_by = 2,
            json_restrictor = "reviews, serpapi_pagination",
            next_page_token = results["serpapi_pagination"]["next_page_token"],    
        )

        

        # Sort by Date in descending order
        if not df.empty:
            df = df.sort_values(by='Date', ascending=False).reset_index(drop=True)   

    return df

In [99]:
df = fetch_reviews ("in.swiggy.android", '2026-01-01', '2026-01-08')
print (df)

            Date                                             Review
0     2026-01-07                                                gud
1     2026-01-07  Worst app...even worse customer service. They ...
2     2026-01-07                                     good very good
3     2026-01-07                                        perfect 👍👍👍
4     2026-01-07                                       yes hai bhai
...          ...                                                ...
3234  2026-01-01                                                  👌
3235  2026-01-01                                 Easy to use portal
3236  2026-01-01                                               nice
3237  2026-01-01                                            nice 👍🏻
3238  2026-01-01  offers too less and only few restaurants only ...

[3239 rows x 2 columns]
